# Testing Adafruit IO API
#### https://adafruit-io-python-client.readthedocs.io/en/latest/index.html

In [1]:
import datetime
from Adafruit_IO import *

In [2]:
import pandas as pd

In [3]:
dStamp=datetime.datetime.now().strftime("%m_%d_%Y")
hashPath = '/Users/cad/simpHashes/csIO.txt'

In [4]:
def connect_REST(hashPath):
    simpHash=open(hashPath)
    a=list(simpHash)
    userName = a[0].strip()
    apiKey = a[1]
    restClient = Client(userName,apiKey)
    return restClient
    
def connect_MQTT(hashPath):
    simpHash=open(hashPath)
    a=list(simpHash)
    userName = a[0].strip()
    apiKey = a[1]
    mqttClient = MQTTClient(userName,apiKey)
    return mqttClient

In [5]:
aio=connect_REST(hashPath)

In [6]:
# check all the feeds
# This will make a list of all feeds.
allFeeds = aio.feeds()

In [7]:
print(allFeeds)

[Feed(name='rig-cadbook', key='rig-cadbook', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60), Feed(name='cad-rig', key='cad-rig', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60), Feed(name='cad-waterconsumed', key='cad-waterconsumed', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60), Feed(name='cad-topvol', key='cad-topvol', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60), Feed(name='cad-weight', key='cad-weight', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60), Feed(name='cad2-rig', key='cad2-rig', description=None, unit_typ

In [8]:
feedList = []
for things in allFeeds:
    feedList.append(things.name)
print(feedList)

['rig-cadbook', 'cad-rig', 'cad-waterconsumed', 'cad-topvol', 'cad-weight', 'cad2-rig', 'cad2-waterconsumed', 'cad2-topvol', 'cad2-weight', 'newFeed', 'cadbook', 'rig-ubuntu']


In [9]:
# let's see what a feed's metadata contains.
print(allFeeds[1])

Feed(name='cad-rig', key='cad-rig', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60)


#### The name is needed to create the feed, but the key is how it is addressed later. By default, the API will respect formating of text, but not the key. So, keep this in mind. 

In [10]:
thisFeed = aio.feeds('cad-rig')
print(thisFeed)

Feed(name='cad-rig', key='cad-rig', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60)


In [11]:
# lets create a feed with an uppercase char, this is ok
feedName = "asciiFeed"
feed = Feed(feedName)
createResult = aio.create_feed(feed)

In [12]:
print(createResult)

Feed(name='asciiFeed', key='asciifeed', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60)


In [13]:
# but to access it, we have to ask for lowercase only, because note above the key is lowercase
thisFeed = aio.feeds(feedName.lower())
print(thisFeed)

Feed(name='asciiFeed', key='asciifeed', description=None, unit_type=None, unit_symbol=None, history=True, visibility='private', license=None, status_notify=False, status_timeout=60)


In [14]:
# now let's delete the feed
feed = aio.delete_feed(feedName.lower())

In [15]:
# now let's get a feed's data
allData = aio.data('cad-waterconsumed')

In [22]:
allData[0]

Data(created_epoch=1541074185, created_at='2018-11-01T12:09:45Z', updated_at=None, value='0.0023', completed_at=None, feed_id=900018, expiration='2018-12-31T12:09:45Z', position=None, id='0E0GBWPMZ6P21VPNKV1KB4Q75N', lat=None, lon=None, ele=None)

In [29]:
# The list of subject feeds are:
# -rig,-task,-waterconsumed,-targetconsumption,-weight,-performance
def makeSubjectFeeds(subjectID,mqttClient):
    subFeedLabels = ['rig','task','waterconsumed','targetconsumption','weight','performance']
    for x in subFeedLabels:
        feedStr = subjectID.lower() + '-' + x
        feed = Feed(name = feedStr)
        try:
            mqttClient.create_feed(feed)
            print("did make feed:{}".format(feedStr))
        except:
            print("did not make feed: {}".format(feedStr))

In [32]:
makeSubjectFeeds('strongsad',aio)

did make feed:strongsad-rig
did make feed:strongsad-task
did make feed:strongsad-waterconsumed
did make feed:strongsad-targetconsumption
did make feed:strongsad-weight
did make feed:strongsad-performance


In [99]:
# Now let's see how we would make a pandas dataframe from this. 
curFeedName = 'cad-waterconsumed'
tCreated_ats=[]
tDates=[]
tTimes=[]
tIDs = []
tValues=[]
tFeedNames = []
for x in allData:
    tCreated_ats.append(x.created_at)
    tS=x.created_at.split('T')
    tDates.append(tS[0])
    tTimes.append(tS[1])
    tIDs.append(x.id)
    tValues.append(float(x.value))
    tFeedNames.append(curFeedName)

tempArray = [tFeedNames,tCreated_ats,tDates,tTimes,tIDs,tValues]
varIndex = ['feed_name','created_at','date','time','id','value']
tempArray=list(list(zip(*tempArray)))
newDF = pd.DataFrame(tempArray,columns = varIndex)
newDF.to_csv('/Users/cad/testFEEDBU.csv')

In [20]:
# this is how we look at feeds containing something (simple list enumeration)
weightFeeds = []
for idx, val in enumerate(feedList):
    if 'weight' in val:
        weightFeeds.append(feedList[idx])
print(weightFeeds)

['cad-weight', 'cad2-weight']


In [21]:
# now we can infer the active animals
activeAnimals = []
for x in weightFeeds:
    tA=x.split('-')
    activeAnimals.append(tA[0])
    
print(activeAnimals)
    

['cad', 'cad2']
